In [1]:
%load_ext jupyter_black

In [2]:
from copy import deepcopy
from collections import Counter
from typing import Dict, Any, Union
from IPython.display import display
from graphviz import Digraph
from scipy.optimize import minimize
from sklearn.datasets import load_iris, make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd

In [3]:
def entropy(y: npt.ArrayLike, weights: npt.ArrayLike = None) -> float:
    """Calculate the entropy of a target array."""
    counts = np.unique(y, return_counts=True)
    if weights is None:
        probs = counts[1] / y.shape[0]
        ans = -np.sum(probs * np.log(probs + 1e-10))
    else:
        ans = 0.0
        total_weight = weights.sum()
        for value in counts[0]:
            prob = weights[y == value].sum() / total_weight
            ans -= prob * np.log(prob + 1e-10)
    return ans


class DecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    """
    Decision tree classifier, which can be trained, can predict class labels(miraculously) and display itself if used in a frontend environment.
    """

    def __init__(
        self,
        max_depth: int = None,
        min_samples_split: int = None,
        min_samples_leaf: int = None,
    ):
        self.max_depth = -1 if max_depth is None else max_depth
        self.min_samples_split = 2 if min_samples_split is None else min_samples_split
        self.min_samples_leaf = 1 if min_samples_leaf is None else min_samples_leaf

    def fit(
        self,
        X: npt.ArrayLike,
        y: npt.ArrayLike,
        features: list = None,
        cat_features: list = None,
        sample_weight: list = None,
    ) -> None:
        """Fit the decision tree to passed data."""

        if features is None:
            self._features = [f"x_{i}" for i in range(X.shape[1])]
        else:
            self._features = features

        if cat_features is None:
            self._cat_features = []
        else:
            self._cat_features = cat_features

        self._weights = sample_weight

        if isinstance(y[0], str):
            self._prediction_dtype = "object"
        else:
            self._prediction_dtype = "int64"

        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]

        self._tree = DecisionTreeClassifier._c45_algorithm(
            X,
            y,
            self._features,
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            min_samples_split=self.min_samples_split,
            cat_features=self._cat_features,
            weights=self._weights,
        )
        self._n_leaves = DecisionTreeClassifier._compute_subtree_leaves(self._tree)

    def predict(self, X: npt.ArrayLike) -> npt.ArrayLike:
        """Predict class labels for given data."""
        ans = np.zeros((X.shape[0],), dtype=self._prediction_dtype)
        for i in range(ans.shape[0]):
            node = self._tree
            while "children" in node:
                feature_id = self._features.index(node["feature"])
                if node["feature"] in self._cat_features:
                    node = node["children"][X[i, feature_id]]
                else:
                    node = (
                        node["children"][f"<={node['threshold']:0.2f}"]
                        if X[i, feature_id] <= node["threshold"]
                        else node["children"][f">{node['threshold']:0.2f}"]
                    )
            ans[i] = node["majority_class"]
        return ans

    # =============================================================================
    # Tree construction
    # =============================================================================

    def _c45_algorithm(
        X: npt.ArrayLike,
        y: npt.ArrayLike,
        features: list,
        max_depth: int,
        min_samples_split: int,
        min_samples_leaf: int,
        cat_features: list,
        weights: list,
    ) -> Dict[str, Any]:
        """Recursively build an C4.5 decision tree."""

        majority_class = DecisionTreeClassifier._majority_class(y)
        error = np.sum(y != majority_class)

        # Base case: all samples same class
        if len(set(y)) == 1 or X.shape[1] < 1:
            return {"majority_class": majority_class, "error": error}

        # Node contains not enough elements to split
        if len(y) < min_samples_split:
            return {"majority_class": majority_class, "error": error}

        # Max tree depth is reached
        if max_depth == 0:
            return {"majority_class": majority_class, "error": error}

        best_id, best_feature, threshold = DecisionTreeClassifier._select_best_feature(
            X, y, features, cat_features, weights
        )
        feature_values = np.unique(X[:, best_id])
        new_features = features.copy()
        new_features.remove(best_feature)

        tree = {
            "feature": best_feature,
            "majority_class": majority_class,
            "children": {},
            "error": error,
        }

        if best_feature in cat_features:
            for value in feature_values:
                mask = X[:, best_id] == value
                X_sub, y_sub, weights_sub = (
                    np.delete(X, best_id, axis=1)[mask],
                    y[mask],
                    weights[mask],
                )
                if len(y_sub) < min_samples_leaf:
                    child_majority_class = DecisionTreeClassifier._majority_class(y_sub)
                    tree["children"][value] = {
                        "majority_class": child_majority_class,
                        "error": np.sum(y_sub != child_majority_class),
                    }
                else:
                    tree["children"][value] = DecisionTreeClassifier._c45_algorithm(
                        X_sub,
                        y_sub,
                        new_features,
                        max_depth=max_depth - 1,
                        min_samples_leaf=min_samples_leaf,
                        min_samples_split=min_samples_split,
                        cat_features=cat_features,
                        weights=weights,
                    )
        else:
            mask = X[:, best_id] <= threshold
            not_mask = np.logical_not(mask)
            if (
                np.sum(mask) >= min_samples_leaf
                and (mask.shape[0] - np.sum(mask)) >= min_samples_leaf
            ):
                tree["threshold"] = threshold
                X_sub, y_sub, weights_sub = (
                    np.delete(X, best_id, axis=1)[mask],
                    y[mask],
                    weights[mask],
                )
                tree["children"][f"<={threshold:0.2f}"] = (
                    DecisionTreeClassifier._c45_algorithm(
                        X_sub,
                        y_sub,
                        new_features,
                        max_depth=max_depth - 1,
                        min_samples_leaf=min_samples_leaf,
                        min_samples_split=min_samples_split,
                        cat_features=cat_features,
                        weights=weights_sub,
                    )
                )

                X_sub, y_sub, weights_sub = (
                    np.delete(X, best_id, axis=1)[not_mask],
                    y[not_mask],
                    weights[not_mask],
                )
                tree["children"][f">{threshold:0.2f}"] = (
                    DecisionTreeClassifier._c45_algorithm(
                        X_sub,
                        y_sub,
                        new_features,
                        max_depth=max_depth - 1,
                        min_samples_leaf=min_samples_leaf,
                        min_samples_split=min_samples_split,
                        cat_features=cat_features,
                        weights=weights_sub,
                    )
                )

        return tree

    def _majority_class(y: npt.ArrayLike):
        return Counter(y).most_common(1)[0][0]

    def _gain_function(
        X: npt.ArrayLike,
        y: npt.ArrayLike,
        feature_idx: int,
        cat_feature: bool,
        weights: npt.ArrayLike,
    ) -> float:
        """Calculate gain for a given feature."""

        if cat_feature is None or cat_feature == False:

            def target_fn(
                theta: float, X: npt.ArrayLike, y: npt.ArrayLike, weights: npt.ArrayLike
            ):
                mask_left = X[:, feature_idx] <= theta
                mask_right = X[:, feature_idx] > theta
                # raise ValueError(
                #     f"mask:{mask_left.shape}\nw:{weights.shape}\ny:{y.shape}\n{weights[mask_left].shape}"
                # )
                if weights is None:
                    return mask_left.sum() / y.shape[0] * entropy(
                        y[mask_left],
                        weights=weights,
                    ) + mask_right.sum() / y.shape[0] * entropy(
                        y[mask_right],
                        weights=weights,
                    )
                else:
                    return mask_left.sum() / y.shape[0] * entropy(
                        y[mask_left],
                        weights=weights[mask_left],
                    ) + mask_right.sum() / y.shape[0] * entropy(
                        y[mask_right],
                        weights=weights[mask_right],
                    )

            target_fn0 = lambda T: target_fn(T, X, y, weights)

            minimize_result = minimize(target_fn0, x0=np.mean(X[:, feature_idx]))
            ans = {"value": minimize_result.fun, "threshold": minimize_result.x[0]}
        else:
            values, counts = np.unique(X[:, feature_idx], return_counts=True)
            probs = counts / y.shape[0]
            entropies = np.array(
                list(
                    map(
                        lambda x: entropy(
                            y[X[:, feature_idx] == x],
                            weights=weights[X[:, feature_idx] == x],
                        ),
                        values,
                    )
                )
            )
            ans = {"value": np.sum(probs * entropies), "threshold": None}

        return ans

    def _select_best_feature(
        X: npt.ArrayLike,
        y: npt.ArrayLike,
        features: list,
        cat_features: list,
        weights: list,
    ) -> list:
        """Select the feature with the highest information gain."""
        gains = [
            DecisionTreeClassifier._gain_function(
                X, y, i, feature in cat_features, weights
            )
            for i, feature in enumerate(features)
        ]
        best_idx = np.argmin(gain["value"] for gain in gains)
        return [best_idx, features[best_idx], gains[best_idx]["threshold"]]

    # =============================================================================
    # Tree pruning
    # =============================================================================

    def get_pruned_tree(self, alpha: float) -> Dict:
        """Prune the tree using cost-complexity pruning with parameter `alpha`."""
        ans = DecisionTreeClassifier(
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            min_samples_split=self.min_samples_split,
        )
        ans._tree = deepcopy(self._tree)
        ans._features = deepcopy(self._features)
        ans._n_leaves = self._n_leaves
        ans._cat_features = deepcopy(self._cat_features)
        ans._prediction_dtype = deepcopy(self._prediction_dtype)
        return ans

    def prune_tree(self, alpha: float) -> None:
        """Prune the underlying decision tree using cost-complexity pruning with predefines `alpha`."""
        self._tree = DecisionTreeClassifier._cost_comprexity_pruning(
            self._tree, alpha, inplace=True
        )

    def _compute_subtree_error(tree: Dict) -> int:
        """Calculate the total misclassification error of a (sub)tree."""

        if not "children" in tree:
            return tree["error"]

        total_error = 0
        for child in tree["children"]:
            total_error += DecisionTreeClassifier._compute_subtree_error(
                tree["children"][child]
            )

        return total_error

    def _compute_subtree_leaves(tree: Dict) -> int:
        """Count the number of leaf nodes in a (sub)tree."""
        if not "children" in tree:
            return 1

        total_leaves = 0
        for child in tree["children"]:
            total_leaves += DecisionTreeClassifier._compute_subtree_leaves(
                tree["children"][child]
            )

        return total_leaves

    def _collect_pruning_candidates(tree: Dict, candidates: list) -> None:
        """Collect non-leaf nodes with their effective alpha values."""
        if not "children" in tree:
            return candidates

        subtree_error = DecisionTreeClassifier._compute_subtree_error(tree)
        complexity_error = DecisionTreeClassifier._compute_subtree_leaves(tree)
        R = tree["error"]
        effective_alpha = (R - subtree_error) / complexity_error

        for child in tree["children"]:
            DecisionTreeClassifier._collect_pruning_candidates(
                tree["children"][child], candidates
            )

        candidates.append((tree, effective_alpha))

        return candidates

    def _cost_comprexity_pruning(self, alpha: float, inplace: bool = None) -> dict:
        if inplace is True:
            tree_to_prune = deepcopy(self._tree)
        else:
            tree_to_prune = self._tree
        while True:
            candidates = []
            candidates = DecisionTreeClassifier._collect_pruning_candidates(
                tree_to_prune, candidates
            )
            candidates.sort(key=lambda x: x[1])

            if not candidates:
                break

            weakest_subtree, weakest_alpha = candidates[0]

            if weakest_alpha > alpha:
                break

            weakest_subtree["children"] = {}
            weakest_subtree.pop("feature")

        return tree_to_prune

    # =============================================================================
    # Tree visualization
    # =============================================================================

    def show_tree(self):
        """Visualize the decision tree."""
        dot = DecisionTreeClassifier._visualize_tree(self._tree, self._features)
        display(dot)

    def _visualize_tree(
        tree: Dict[str, Any],
        feature_names: list,
        dot: Digraph = None,
        parent: str = None,
        edge_label: str = None,
    ) -> Digraph:
        """Recursively visualize the decision tree using Graphviz."""
        if dot is None:
            dot = Digraph(comment="Decision Tree")

        # Create a unique node ID
        node_id = str(id(tree))

        # Add the current node
        if not "children" in tree:
            node_label = f"Class: {tree['majority_class']}\nError: {tree['error']}"
        else:
            node_label = f"Feature: {tree['feature']}\nError: {tree['error']}"
        dot.node(node_id, node_label)

        # Connect to parent node if exists
        if parent is not None:
            dot.edge(parent, node_id, label=edge_label)

        # Recursively add children
        if "children" in tree:
            for value, child in tree["children"].items():
                DecisionTreeClassifier._visualize_tree(
                    child, feature_names, dot, node_id, str(value)
                )

        return dot

    # =============================================================================
    # Utility functions
    # =============================================================================

In [4]:
class AdaBoostClassifierScratch(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator: object, n_estimators: int = None):
        self.n_estimators = 50 if n_estimators is None else n_estimators
        self.estimator = DecisionTreeClassifier if estimator is None else estimator
        self.classifier_weights_ = np.zeros(self.n_estimators)
        self.classifiers_ = [self.estimator for i in range(self.n_estimators)]

    def fit(self, X: npt.ArrayLike, y: npt.ArrayLike):
        n_samples = X.shape[0]
        self.classes_ = np.unique(y)
        self.classifier_sample_weights_ = np.zeros((self.n_estimators, n_samples))
        self.classifier_sample_weights_[0, :] = np.ones(n_samples) / n_samples

        for i in range(self.n_estimators):
            self.classifiers_[i].fit(
                X, y, sample_weight=self.classifier_sample_weights_[i, :]
            )
            print(f"#{i+1}...OK!")
            preds = self.classifiers_[i].predict(X)
            errors_mask = preds != y

            # the error term is artificially increased so that the method continues working if classifier does perfect classification
            classifier_error = (
                np.sum(self.classifier_sample_weights_[i, errors_mask])
                / self.classifier_sample_weights_[i, :].sum()
            ) + 1e-10

            print(f"Error term: {classifier_error:0.2f}")
            if classifier_error < 1.0 - 1.0 / self.classes_.shape[0]:
                self.classifier_weights_[i] = (
                    np.log(1.0 - classifier_error)
                    - np.log(classifier_error)
                    + np.log(self.classes_.shape[0] - 1.0)
                )
                if i < self.n_estimators - 1:
                    self.classifier_sample_weights_[i + 1, :] = (
                        self.classifier_sample_weights_[i, :]
                        * np.exp(self.classifier_weights_[i] * errors_mask)
                    )

                    # Normalize the weights
                    self.classifier_sample_weights_[i + 1, :] = (
                        self.classifier_sample_weights_[i + 1, :]
                        / self.classifier_sample_weights_[i + 1, :].sum()
                    )
            else:
                print("Bad error value. Resetting the weights...")
                self.classifier_weights_[i] = 0.0
                if i < self.n_estimators - 1:
                    self.classifier_sample_weights_[i + 1, :] = (
                        np.ones(n_samples) / n_samples
                    )

    def predict(self, X):
        cls_predictions = np.zeros((self.n_estimators, X.shape[0]))
        for cls_num, cls in enumerate(self.classifiers_):
            cls_predictions[cls_num, :] = cls.predict(X)
        predictions = np.zeros(X.shape[0])
        for i in range(predictions.shape[0]):
            predictions[i] = self.classes_[
                np.argmax(
                    [
                        (
                            self.classifier_weights_
                            * (cls_predictions[:, i] == pred_class)
                        ).sum(axis=0)
                        for pred_class in self.classes_
                    ]
                )
            ]
        return predictions

In [5]:
# find weights for scatter plot
# def get_sample_weights_per_stage(ada, X, y):
#     """Recalculate sample weights at each stage of boosting"""
#     n_samples = len(X)
#     weights = np.ones(n_samples) / n_samples
#     stage_weights = []

#     for estimator, alpha in zip(ada.estimators_, ada.estimator_weights_):
#         stage_weights.append(weights.copy())
#         pred = estimator.predict(X)
#         incorrect = (y != pred).astype("float")
#         weights *= np.exp(alpha * incorrect)
#         weights = weights / weights.sum()

#     return stage_weights


# stage_weights = get_sample_weights_per_stage(ada, X_train, y_train)
# stage_weights
# stage_weights = [np.ones_like(y) for clf in ada.estimators_]

In [6]:
# def plot_stages(estimator, weights_per_stage):
#     n_estimators = len(estimator.estimators_)
#     fig, axes = plt.subplots(n_estimators, 2, figsize=(12, 4 * n_estimators))
#     xx, yy = np.meshgrid(np.linspace(-2, 3, 300), np.linspace(-2, 2, 300))
#     X_grid = np.c_[xx.ravel(), yy.ravel()]

#     cumulative_pred = np.zeros_like(xx)
#     for i in range(n_estimators):
#         ax_left, ax_right = axes[i]
#         tree = estimator.estimators_[i]
#         weight = estimator.estimator_weights_[i]
#         sample_weights = weights_per_stage[i]
#         point_sizes = 200 * sample_weights / np.max(sample_weights)
#         pred_grid = tree.predict(X_grid).reshape(xx.shape)
#         cumulative_pred += weight * pred_grid

#         # -- Left: cumulative raw score as heatmap + boundary --
#         im = ax_left.contourf(
#             xx, yy, cumulative_pred, levels=100, cmap="RdBu", alpha=0.5
#         )
#         ax_left.contour(
#             xx, yy, cumulative_pred, levels=[0], colors="k", linewidths=1.2
#         )  # decision boundary
#         ax_left.scatter(
#             X_train[:, 0],
#             X_train[:, 1],
#             c=y_train,
#             s=point_sizes,
#             edgecolors="black",
#             # cmap="bwr",
#         )
#         ax_left.set_title(f"Cumulative after {i+1} learners\nWeight = {weight:.2f}")

#         # -- Right: single estimator prediction --
#         ax_right.contourf(xx, yy, pred_grid, levels=100, cmap="RdBu", alpha=0.5)
#         ax_right.contour(xx, yy, pred_grid, levels=[0], colors="k", linewidths=1.2)
#         ax_right.scatter(
#             X_train[:, 0],
#             X_train[:, 1],
#             c=y_train,
#             s=point_sizes,
#             edgecolors="black",
#             # cmap="RdBu",
#         )
#         ax_right.set_title(f"Estimator {i+1} prediction\nWeight = {weight:.2f}")
#         # break
#     plt.tight_layout()
#     plt.show()


# plot_stages(ada, stage_weights)

In [7]:
# # Generate 2D data
# X, y = make_moons(n_samples=300, noise=0.25, random_state=42)
# y = 2 * y - 1  # Convert to {-1, 1}
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
ada1 = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=4,
)
ada1.fit(X_train, y_train)
y_pred_sklearn1 = ada1.predict(X_test)
accuracy_score(y_test, y_pred_sklearn1)

0.9333333333333333

In [9]:
ada2 = AdaBoostClassifier(
    estimator=SklearnDecisionTreeClassifier(),
    n_estimators=4,
)
ada2.fit(X_train, y_train)
y_pred_sklearn2 = ada2.predict(X_test)
accuracy_score(y_test, y_pred_sklearn2)

1.0

In [10]:
scratch_boost1 = AdaBoostClassifierScratch(
    estimator=DecisionTreeClassifier(),
    n_estimators=4,
)
scratch_boost1.fit(X_train, y_train)
y_pred1 = scratch_boost1.predict(X_test)
accuracy_score(y_test, y_pred1)

#1...OK!
Error term: 0.06
#2...OK!
Error term: 0.67
#3...OK!
Error term: 0.67
Bad error value. Resetting the weights...
#4...OK!
Error term: 0.06


0.9333333333333333

In [11]:
scratch_boost2 = AdaBoostClassifierScratch(
    estimator=SklearnDecisionTreeClassifier(),
    n_estimators=4,
)
scratch_boost2.fit(X_train, y_train)
y_pred2 = scratch_boost2.predict(X_test)
accuracy_score(y_test, y_pred2)

#1...OK!
Error term: 0.00
#2...OK!
Error term: 0.00
#3...OK!
Error term: 0.00
#4...OK!
Error term: 0.00


1.0